In [1]:
import os
import pickle
os.chdir('/kaggle/input/kaggle-competition-code/automatic-essay-scoring-2')

from datamodule.preprocessing import *
from model.metric import *
os.chdir('/kaggle/working')

2024-06-25 19:54:52.450312: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-25 19:54:52.450418: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-25 19:54:52.627133: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import polars as pl

test_df = pl.read_csv('/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv')
test_df.shape

(3, 2)

In [3]:
model_directory = '/kaggle/input/deberta-training-example'
model_id = 'deberta-v3-small_fold_{fold_index}'
llm_df = []

for fold_index in range(5):
    model_path = os.path.join(model_directory, model_id.format(fold_index=fold_index))
    llm_df.append(create_llm_features(test_df, model_path))

essay_ids = test_df['essay_id'].to_pandas()
llm_df = pd.concat(llm_df + [essay_ids], axis=1)
llm_df.head()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

,/kaggle/input/deberta-training-example/deberta-v3-small_fold_0_label_0,/kaggle/input/deberta-training-example/deberta-v3-small_fold_1_label_0,/kaggle/input/deberta-training-example/deberta-v3-small_fold_2_label_0,/kaggle/input/deberta-training-example/deberta-v3-small_fold_3_label_0,/kaggle/input/deberta-training-example/deberta-v3-small_fold_4_label_0,essay_id
0,1.337984,1.402706,0.910786,1.157639,0.956461,000d118
1,1.775424,2.029669,1.919433,2.041273,1.949008,000fe60
2,3.398521,3.368047,3.679340,3.384857,3.397962,001ab80


In [4]:
word_feature_df = create_word_features(test_df)
word_feature_df.head()

sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
/kaggle/input/kaggle-competition-code/automatic-essay-scoring-2/datamodule/preprocessing.py:140: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df = df.with_columns(


,essay_id,word_length_greater_equal_1,word_length_greater_equal_2,word_length_greater_equal_3,word_length_greater_equal_4,word_length_greater_equal_5,word_length_greater_equal_6,word_length_greater_equal_7,word_length_greater_equal_8,word_length_greater_equal_9,...,word_length_greater_equal_12,word_length_greater_equal_13,word_length_greater_equal_14,word_length_greater_equal_15,word_length_max,word_length_mean,word_length_std,word_length_q1,word_length_q2,word_length_q3
0,000d118,491,471,391,289,169,119,76,46,32,...,6,6,5,2,25,4.378819,2.538495,3.0,4.0,5.0
1,000fe60,332,311,248,184,103,63,41,29,16,...,0,0,0,0,11,4.012048,2.060968,2.0,4.0,5.0
2,001ab80,550,539,445,328,200,135,103,72,54,...,14,10,5,2,15,4.574545,2.604621,3.0,4.0,5.0


In [5]:
paragraph_feature_df = create_paragraph_features(test_df)
paragraph_feature_df.head()

/kaggle/input/kaggle-competition-code/automatic-essay-scoring-2/datamodule/preprocessing.py:76: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df = df.with_columns(
/kaggle/input/kaggle-competition-code/automatic-essay-scoring-2/datamodule/preprocessing.py:81: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df = df.with_columns(
/kaggle/input/kaggle-competition-code/automatic-essay-scoring-2/datamodule/preprocessing.py:81: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df = df.with_columns(
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. S

,essay_id,paragraph_num_character_greater_equal_0,paragraph_num_character_greater_equal_50,paragraph_num_character_greater_equal_75,paragraph_num_character_greater_equal_100,paragraph_num_character_greater_equal_125,paragraph_num_character_greater_equal_150,paragraph_num_character_greater_equal_175,paragraph_num_character_greater_equal_200,paragraph_num_character_greater_equal_250,...,paragraph_num_sentences_q3,paragraph_num_errors_max,paragraph_num_errors_min,paragraph_num_errors_mean,paragraph_num_errors_sum,paragraph_num_errors_first,paragraph_num_errors_last,paragraph_num_errors_kurtosis,paragraph_num_errors_q1,paragraph_num_errors_q3
0,000d118,1,1,1,1,1,1,1,1,1,...,14.0,23,23,23.00,23,23,23,NaN,23.0,23.0
1,000fe60,5,5,5,5,5,5,5,4,3,...,5.0,1,0,0.80,4,1,1,0.250000,1.0,1.0
2,001ab80,4,4,4,4,4,4,4,4,4,...,8.0,2,0,1.25,5,0,2,-1.371901,1.0,2.0


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

def tfidf_tokenizer(x):
    return x

def tfidf_preprocessor(x):
    return x

with open('/kaggle/input/lgbm-train-notebook/tfidf_vectorizer.pkl', 'rb') as infile:
    tfidf_vectorizer = pickle.load(infile)

tfidf_df = create_tfidf_features(test_df, tfidf_vectorizer)
tfidf_df.head()

/kaggle/input/kaggle-competition-code/automatic-essay-scoring-2/datamodule/preprocessing.py:173: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  essays = df['full_text'].map_elements(sanitize_text)


,tfidf_0,tfidf_1,tfidf_2,tfidf_3,tfidf_4,tfidf_5,tfidf_6,tfidf_7,tfidf_8,tfidf_9,...,tfidf_19251,tfidf_19252,tfidf_19253,tfidf_19254,tfidf_19255,tfidf_19256,tfidf_19257,tfidf_19258,tfidf_19259,essay_id
0,0.0,0.013906,0.0,0.027957,0.000000,0.0,0.000000,0.0,0.0,0.014791,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000d118
1,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.049895,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000fe60
2,0.0,0.000000,0.0,0.000000,0.031504,0.0,0.031526,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,001ab80


In [7]:
from sklearn.feature_extraction.text import CountVectorizer

def count_tokenizer(x):
    return x

def count_preprocessor(x):
    return x

with open('/kaggle/input/lgbm-train-notebook/count_vectorizer.pkl', 'rb') as infile:
    count_vectorizer = pickle.load(infile)
    
bow_df = create_bag_of_words_features(test_df, count_vectorizer)
bow_df.head()

/kaggle/input/kaggle-competition-code/automatic-essay-scoring-2/datamodule/preprocessing.py:185: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  essays = df['full_text'].map_elements(sanitize_text)


,bow_0,bow_1,bow_2,bow_3,bow_4,bow_5,bow_6,bow_7,bow_8,bow_9,...,bow_1969,bow_1970,bow_1971,bow_1972,bow_1973,bow_1974,bow_1975,bow_1976,bow_1977,essay_id
0,4,0,1,0,1,0,0,4,1,0,...,3,0,0,0,0,0,0,0,0,000d118
1,4,0,0,0,0,0,0,0,0,0,...,2,0,0,1,1,0,0,0,0,000fe60
2,6,0,0,0,0,0,0,0,0,0,...,1,0,2,0,0,0,0,0,0,001ab80


In [8]:
test_df = test_df.to_pandas()
for df in [llm_df, paragraph_feature_df, word_feature_df, tfidf_df, bow_df]:
    test_df = test_df.merge(df, on='essay_id', how='left')

test_df.head()

,essay_id,full_text,/kaggle/input/deberta-training-example/deberta-v3-small_fold_0_label_0,/kaggle/input/deberta-training-example/deberta-v3-small_fold_1_label_0,/kaggle/input/deberta-training-example/deberta-v3-small_fold_2_label_0,/kaggle/input/deberta-training-example/deberta-v3-small_fold_3_label_0,/kaggle/input/deberta-training-example/deberta-v3-small_fold_4_label_0,paragraph_num_character_greater_equal_0,paragraph_num_character_greater_equal_50,paragraph_num_character_greater_equal_75,...,bow_1968,bow_1969,bow_1970,bow_1971,bow_1972,bow_1973,bow_1974,bow_1975,bow_1976,bow_1977
0,000d118,Many people have car where they live. The thin...,1.337984,1.402706,0.910786,1.157639,0.956461,1,1,1,...,3,3,0,0,0,0,0,0,0,0
1,000fe60,I am a scientist at NASA that is discussing th...,1.775424,2.029669,1.919433,2.041273,1.949008,5,5,5,...,2,2,0,0,1,1,0,0,0,0
2,001ab80,People always wish they had the same technolog...,3.398521,3.368047,3.679340,3.384857,3.397962,4,4,4,...,3,1,0,2,0,0,0,0,0,0


In [9]:
import lightgbm as lgb
feature_names = list(filter(lambda x: x not in ['full_text', 'essay_id','score'], test_df.columns))
X = test_df[feature_names].astype(np.float32).values

predictions = []
for i in range(5):
    model = lgb.Booster(model_file=f'/kaggle/input/lgbm-train-notebook/fold_{i}.txt')
    prediction = model.predict(X) + a
    predictions.append(prediction)
    
predictions = np.mean(predictions, axis=0)
predictions = np.round(predictions.clip(1, 6))

In [10]:
submission = pd.read_csv("/kaggle/input/learning-agency-lab-automated-essay-scoring-2/sample_submission.csv")
submission['score'] = predictions
submission['score'] = submission['score'].astype(int)
submission.to_csv("submission.csv", index=None)

submission.head()

,essay_id,score
0,000d118,2
1,000fe60,3
2,001ab80,4
